# Exploring presidential candidates endorsements

In [2]:
import json

## Reading and parsing the JSON file

In [8]:
with open(r"../data/parrainages.json", encoding="utf-8") as f:
    ENDORSEMENTS = json.load(f)

In [5]:
ENDORSEMENTS[0]

{'Civilite': 'M.',
 'Nom': 'CORDIVAL',
 'Prenom': 'Gilles',
 'Mandat': 'Maire',
 'Circonscription': 'Mont-Saint-Père',
 'Departement': 'Aisne',
 'Candidat': 'ARTHAUD Nathalie',
 'DatePublication': '2022-02-01T00:00:00'}

In [10]:
len(ENDORSEMENTS)

13427

## Find information about candidates

### Distinct number of candidates

In [15]:
distinct_candidates = set()

for endorsement in ENDORSEMENTS:
    candidat = endorsement['Candidat']
    distinct_candidates.add(candidat)

len(distinct_candidates)

64

### Endorsements per candidates (the hard way)

In [24]:
endorsement_per_candidate = {}

for endorsement in ENDORSEMENTS:
    candidat = endorsement['Candidat']

    if candidat not in endorsement_per_candidate:
        endorsement_per_candidate[candidat] = 1
    else:
        endorsement_per_candidate[candidat] += 1

len(endorsement_per_candidate)

64

In [49]:
relevant_candidates = []

for name, count in endorsement_per_candidate.items():
    if count >= 400:
        relevant_candidates.append((name, count))

len(relevant_candidates)

12

In [50]:
sorted(relevant_candidates, key=lambda record: record[1], reverse=True)

[('PÉCRESSE Valérie', 2636),
 ('MACRON Emmanuel', 2098),
 ('HIDALGO Anne', 1440),
 ('MÉLENCHON Jean-Luc', 906),
 ('ZEMMOUR Éric', 741),
 ('JADOT Yannick', 712),
 ('LASSALLE Jean', 642),
 ('ROUSSEL Fabien', 626),
 ('LE PEN Marine', 622),
 ('DUPONT-AIGNAN Nicolas', 600),
 ('POUTOU Philippe', 596),
 ('ARTHAUD Nathalie', 576)]

### Endorsements per candidates (the lazy way)

In [51]:
from collections import Counter

In [55]:
counter = Counter()
counter["i do not exists"]

0

In [57]:
endorsement_per_candidate = Counter()

for endorsement in ENDORSEMENTS:
    endorsement_per_candidate[endorsement["Candidat"]] += 1

len(endorsement_per_candidate)

64

In [60]:
for name, count in endorsement_per_candidate.most_common():
    if count < 500:
        break
    print(name, count)

PÉCRESSE Valérie 2636
MACRON Emmanuel 2098
HIDALGO Anne 1440
MÉLENCHON Jean-Luc 906
ZEMMOUR Éric 741
JADOT Yannick 712
LASSALLE Jean 642
ROUSSEL Fabien 626
LE PEN Marine 622
DUPONT-AIGNAN Nicolas 600
POUTOU Philippe 596
ARTHAUD Nathalie 576


### Finding the candidates who did run

In [61]:
candidates_who_ran = set()

for name, count in endorsement_per_candidate.most_common():
    if count < 500:
        break

    candidates_who_ran.add(name)

candidates_who_ran

{'ARTHAUD Nathalie',
 'DUPONT-AIGNAN Nicolas',
 'HIDALGO Anne',
 'JADOT Yannick',
 'LASSALLE Jean',
 'LE PEN Marine',
 'MACRON Emmanuel',
 'MÉLENCHON Jean-Luc',
 'POUTOU Philippe',
 'PÉCRESSE Valérie',
 'ROUSSEL Fabien',
 'ZEMMOUR Éric'}

### Analyzing mandates

In [68]:
distinct_mandates = set()

for endorsement in ENDORSEMENTS:
    distinct_mandates.add(endorsement['Mandat'])

# Same as the above
distinct_mandates = set(endorsement['Mandat'] for endorsement in ENDORSEMENTS)
distinct_mandates

{'Conseiller de Paris',
 'Conseiller départemental',
 'Conseiller métropolitain de Lyon',
 'Conseiller régional',
 "Conseiller à l'Assemblée des Français de l'étranger",
 'Conseillère de Paris',
 'Conseillère départementale',
 'Conseillère métropolitaine de Lyon',
 'Conseillère régionale',
 "Conseillère à l'Assemblée des Français de l'étranger",
 'Député',
 'Députée',
 'Maire',
 "Maire d'arrondissement",
 "Maire délégué d'une commune associée ou d'une commune déléguée",
 "Maire déléguée d'une commune associée ou d'une commune déléguée",
 "Membre d'une assemblée d'une collectivité territoriale d'outre-mer à statut particulier",
 "Membre de l'Assemblée de Corse",
 "Président d'un EPCI à fiscalité propre",
 'Président de la Polynésie française',
 'Président du Conseil exécutif de Martinique',
 'Président du conseil consulaire',
 'Président du gouvernement de la Nouvelle-Calédonie',
 "Présidente d'un EPCI à fiscalité propre",
 'Présidente du conseil consulaire',
 'Représentant français au 

### Filtering the relevant endorsements

In [71]:
RELEVANT_ENDORSEMENTS = []

for endorsement in ENDORSEMENTS:
    if (
        endorsement["Candidat"] in candidates_who_ran and
        endorsement["Mandat"].startswith("Maire")
    ):
        RELEVANT_ENDORSEMENTS.append(endorsement)

len(RELEVANT_ENDORSEMENTS) / len(ENDORSEMENTS)

0.6045281894689805